In [1]:
# Define Node Classes
class Node:
    pass

class UnaryOperator(Node):
    def __init__(self, operand):
        self.operand = operand

class BinaryOperator(Node):
    def __init__(self, left, right):
        self.left = left
        self.right = right

class Add(BinaryOperator):
    pass

class Sub(BinaryOperator):
    pass

class Mul(BinaryOperator):
    pass

class Div(BinaryOperator):
    pass

class Mod(BinaryOperator):  # New Modulo Operator
    pass

class Negate(UnaryOperator):
    pass

class Number(Node):
    def __init__(self, value):
        self.value = value

# Base Visitor
class NodeVisitor:
    def visit(self, node):
        methname = 'visit_' + type(node).__name__
        meth = getattr(self, methname, None)
        if meth is None:
            meth = self.generic_visit
        return meth(node)
    
    def generic_visit(self, node):
        raise RuntimeError(f'No visit_{type(node).__name__} method')

# Evaluator
class Evaluator(NodeVisitor):
    def visit_Number(self, node):
        return node.value
    
    def visit_Add(self, node):
        return self.visit(node.left) + self.visit(node.right)
    
    def visit_Sub(self, node):
        return self.visit(node.left) - self.visit(node.right)
    
    def visit_Mul(self, node):
        return self.visit(node.left) * self.visit(node.right)
    
    def visit_Div(self, node):
        return self.visit(node.left) / self.visit(node.right)
    
    def visit_Mod(self, node):  # Modulo support
        return self.visit(node.left) % self.visit(node.right)
    
    def visit_Negate(self, node):
        return -self.visit(node.operand)

# StackCode Generator
class StackCode(NodeVisitor):
    def generate_code(self, node):
        self.instructions = []
        self.visit(node)
        return self.instructions
    
    def visit_Number(self, node):
        self.instructions.append(('PUSH', node.value))
    
    def binop(self, node, instruction):
        self.visit(node.left)
        self.visit(node.right)
        self.instructions.append((instruction,))
    
    def visit_Add(self, node):
        self.binop(node, 'ADD')
    
    def visit_Sub(self, node):
        self.binop(node, 'SUB')
    
    def visit_Mul(self, node):
        self.binop(node, 'MUL')
    
    def visit_Div(self, node):
        self.binop(node, 'DIV')
    
    def visit_Mod(self, node):  # Modulo instruction
        self.binop(node, 'MOD')
    
    def unaryop(self, node, instruction):
        self.visit(node.operand)
        self.instructions.append((instruction,))
    
    def visit_Negate(self, node):
        self.unaryop(node, 'NEG')

# Simplifier
class Simplifier(NodeVisitor):
    def visit_Number(self, node):
        return node  # Numbers remain unchanged
    
    def visit_Add(self, node):
        left = self.visit(node.left)
        right = self.visit(node.right)
        if isinstance(left, Number) and isinstance(right, Number):
            return Number(left.value + right.value)
        return Add(left, right)
    
    def visit_Sub(self, node):
        left = self.visit(node.left)
        right = self.visit(node.right)
        if isinstance(left, Number) and isinstance(right, Number):
            return Number(left.value - right.value)
        return Sub(left, right)
    
    def visit_Mul(self, node):
        left = self.visit(node.left)
        right = self.visit(node.right)
        if isinstance(left, Number) and isinstance(right, Number):
            return Number(left.value * right.value)
        return Mul(left, right)
    
    def visit_Div(self, node):
        left = self.visit(node.left)
        right = self.visit(node.right)
        if isinstance(left, Number) and isinstance(right, Number):
            return Number(left.value / right.value)
        return Div(left, right)

    def visit_Negate(self, node):
        operand = self.visit(node.operand)
        if isinstance(operand, Number):
            return Number(-operand.value)
        return Negate(operand)

# Example Expression: 1 + 2 * (3 - 4) / 5
t1 = Sub(Number(3), Number(4))
t2 = Mul(Number(2), t1)
t3 = Div(t2, Number(5))
t4 = Add(Number(1), t3)

# Evaluator Example
evaluator = Evaluator()
print("Evaluation Result:", evaluator.visit(t4))  # Output: 0.6

# StackCode Example
stack = StackCode()
print("Stack Code:", stack.generate_code(t4))
# Output: [('PUSH', 1), ('PUSH', 2), ('PUSH', 3), ('PUSH', 4), ('SUB',), ('MUL',), ('PUSH', 5), ('DIV',), ('ADD',)]

# Simplifier Example
simplifier = Simplifier()
simplified = simplifier.visit(t4)
print("Simplified Expression:", evaluator.visit(simplified))  # Output: 0.6


Evaluation Result: 0.6
Stack Code: [('PUSH', 1), ('PUSH', 2), ('PUSH', 3), ('PUSH', 4), ('SUB',), ('MUL',), ('PUSH', 5), ('DIV',), ('ADD',)]
Simplified Expression: 0.6
